In [1]:
import pandas as pd
import requests
from apikey import apikey 
%load_ext blackcellmagic

In [2]:
# load and transform 2019 data
df_soil_19 = pd.read_excel("2019_Metals_Results_-_Soil_Kitchen.xls")

df_soil_19 = df_soil_19.drop(columns=["Unnamed: 0"])

df_soil_19.columns = df_soil_19.iloc[0]

df_base_19 = df_soil_19.head()

df_soil_19 = df_soil_19.drop(index=[0, 1, 2, 3, 4])

df_soil_19 = df_soil_19[["Street", "Cross Street", "Lead", "Arsenic"]]

In [3]:
# Load and transform 2015-2018 data
df_soil_18_15 = pd.read_csv("2015-18 Soil_Kitchen_Log.csv")

df_soil_18_15 = df_soil_18_15[
    ["Street", "Cross Street", "Full Address", "Lead", "Arsenic"]
]

In [4]:
def request_geocoder(intersection, apikey, city="Austin, TX"):
    """returns lat. lon for cross streets in street1@street2 format.
    Defailt city is Austin, TX and apikey is required"""

    base_url = "https://geocoder.ls.hereapi.com/6.2/geocode.json"
    parameters = {"city": city, "street": intersection, "gen": 9, "apiKey": apikey}
    try:
        response = requests.get(base_url, params=parameters)
        j_resp = response.json()
        lat = j_resp["Response"]["View"][0]["Result"][0]["Location"]["DisplayPosition"][
            "Latitude"
        ].astype(float)
        lon = j_resp["Response"]["View"][0]["Result"][0]["Location"]["DisplayPosition"][
            "Longitude"
        ].astype(float)
    #         print(parameters['street'], 'success')
    except:
        lat = ""
        lon = ""
        print(parameters["street"], response)

    return pd.Series([lat, lon], index=["Lat", "Lon"])

In [5]:
def query_cross_streets(df):
    """this function uses geocoder.ls.hereapi.com to find the geocodes for cross streets from a dataframe"""
    df[["Lat", "Lon"]] = df.apply(
        lambda x: request_geocoder(x["Intersection"], apikey), axis=1
    )
    return df

In [6]:
df_soil_18_15["Lead"] = pd.to_numeric(df_soil_18_15["Lead"], errors="coerce")

df_soil_18_15["Arsenic"] = pd.to_numeric(df_soil_18_15["Arsenic"], errors="coerce")

df_soil = df_soil_19.append(df_soil_18_15, ignore_index=True)

df_soil["Intersection"] = df_soil.apply(
    lambda x: str(x["Street"]) + "@" + str(x["Cross Street"]), axis=1
)

# fillna with median values to preserve dataset size
df_soil = df_soil.fillna(df_soil.median())

df_soil_mean = (
    df_soil[["Intersection", "Lead", "Arsenic"]]
    .groupby(["Intersection"])
    .mean()
    .reset_index()
)

# print(df_soil)

In [7]:
# geocode data
df_soil_mean_loc = query_cross_streets(df_soil_mean)

10th @ Menden <Response [200]>
10th @ Menden <Response [200]>
10th @ Prospect -- Back <Response [200]>
10th @ Prospect -- Front <Response [200]>
10th St.@Theresa Ave <Response [200]>
10th@Northwestern <Response [200]>
12th St.@Meridian Lane <Response [200]>
12th@Chicon <Response [200]>
12th@EM Franklan <Response [200]>
12th@Waller <Response [200]>
13th @ Singleton Ave. <Response [200]>
1458@Davis Ln <Response [200]>
14th @ Comal <Response [200]>
14th@Poquito <Response [200]>
14th@Salina <Response [200]>
15th st@Pease <Response [200]>
16th@Deloney St. <Response [200]>
17th@Chicon <Response [200]>
1st @ Stassney <Response [200]>
21st@Salina St. <Response [200]>
22nd @ Chicon <Response [200]>
28 1/2 ST@Salado St <Response [200]>
28th @ Layafette Ave. <Response [200]>
290E@973 <Response [200]>
2nd@West Ave <Response [200]>
30th@Kerbey <Response [200]>
30th@Lamar <Response [200]>
32nd@French <Response [200]>
39th @ Ave F <Response [200]>
39th @ Red River <Response [200]>
42nd@Bellevue <Resp

Colony Loop Dr@Valley Field Dr <Response [200]>
Comal @ Concho St. <Response [200]>
Connecticut Dr.@Jamestown <Response [200]>
Connor @Cambourne <Response [200]>
Convict Hill Rd.@Escarpment Blvd. <Response [200]>
Convict Hill@Whistlestop <Response [200]>
Cooper@Bodark Ln <Response [200]>
Copano@Eskew <Response [200]>
Coppermead@Edenbourgh <Response [200]>
Cornflower@Archaleta <Response [200]>
Coronado @Mathias <Response [200]>
Country Ln@Morning Sun Ln <Response [200]>
Crenata@Jollyville <Response [200]>
Creole Dr. @Lamplight Village <Response [200]>
Creston Lane@Northway Drive <Response [200]>
Crestwind@Western Sky <Response [200]>
Crown Ct@Villa Park <Response [200]>
Crystal Brook@Milrace <Response [200]>
Cushing Dr@School Days Dr <Response [200]>
DG Collins@Hwy 183 <Response [200]>
Davis Lane @ Star Cross <Response [200]>
Davis Ln@West Gate Blvd <Response [200]>
Decker Ln@Duffan <Response [200]>
Deen@N. Lamar <Response [200]>
Deer Track@Maverick Dr <Response [200]>
Delores Ave @Maso

Lafayette @ 30th <Response [200]>
Laird@Pequeno <Response [200]>
Lake Austin @ Mopac <Response [200]>
Lake Austin @Veterans Dr <Response [200]>
Lake Pines@Longdraw <Response [200]>
Lakeline @ Hwy 183 <Response [200]>
Lakeline Blvd @Cypress Creek Blvd <Response [200]>
Lakeshore@Lady Bird Lake <Response [200]>
Land Grant Way@Cotton Gin <Response [200]>
Langston@Burnell <Response [200]>
Langston@Loyola <Response [200]>
Larkwood Dr@Berkman Dr.  <Response [200]>
Lauralan@Claxton <Response [200]>
Layola Ln@Dubuque Ln <Response [200]>
Leeward@Palisade <Response [200]>
Lemens Spice Trail@4 Iron Dr.  <Response [200]>
Lemens Spice Trail@Four Irons <Response [200]>
Limon Ln@S. 4th St. <Response [200]>
Linda@Wheless <Response [200]>
Live Oak Street@East Side Drive <Response [200]>
Live Oak@Schriber <Response [200]>
Lone Oak Tr@Jones Rd <Response [200]>
Longwood@Tall Oaks <Response [200]>
Loop 1@Enfield <Response [200]>
Lorrain@13th  <Response [200]>
Lott@Nickols <Response [200]>
Lovely Ln@Yucca Hi

S. Mary Moore Searight@Slaughter Lane <Response [200]>
S. Redondo @ Meander <Response [200]>
S. Sledge@3rd St <Response [200]>
SE Inner Loop@Rockride Ln <Response [200]>
Saddle @ Black Angus <Response [200]>
Sage Hollow@Cricket Hollow <Response [200]>
Sahara@Libyan <Response [200]>
Salem Walk Dr@Cedar Glen <Response [200]>
San Lucas Ln@Kavai ln <Response [200]>
Sandy Bottom @Blue Pond Dr <Response [200]>
Santa Cruz Dr@Duval <Response [200]>
Santa Maria @Matamoros <Response [200]>
Santa Maria@Pedernales <Response [200]>
Sara Dr@Ebert Ave <Response [200]>
SawyerFay Ln@Jack Ryan Ln <Response [200]>
Scales @Mattie <Response [200]>
Scales@Mathe <Response [200]>
Scenic Brook Dr@Grove Crest  <Response [200]>
Schriber @ Live Oak <Response [200]>
Scofield Ln@Rampart St <Response [200]>
Scotland Well @ Caira Cove <Response [200]>
Scotland Well Dr@Spicewoods Springs Rd <Response [200]>
Scotland Well Drive@Tweedsmuir Drive <Response [200]>
Sendero Hill@Alsace Tr <Response [200]>
Sentinel Hill @Her

In [8]:
# convert to float64
df_soil_mean_loc['Lat'] = pd.to_numeric(df_soil_mean_loc['Lat'])
df_soil_mean_loc['Lon'] = pd.to_numeric(df_soil_mean_loc['Lon'])

In [9]:
# df_soil_mean_loc = df_soil_mean_loc.fillna(df_soil_mean_loc.median())

In [10]:
# df_soil_mean_loc.to_csv('df_soil_mean_loc.csv')